In [1]:
import pandas as pd
import geopandas as gpd
import os
import re
print(os.getcwd())
os.chdir(r"C:\Users\LMENENDEZ\GitHub\MultiModX")
print(os.getcwd())
pd.set_option('display.max_columns', None)

c:\Users\LMENENDEZ\GitHub\MultiModX\notebooks
C:\Users\LMENENDEZ\GitHub\MultiModX


NOTE: I might want to remove the legs column in the future 

Some characteristics of the trips that go abroad:
- if the person is spanish: we know the "roaming out", i.e., the signal they emitted when they connect to roaming. Hence we know with certainty the country where they arrived. However, if they did a connection, we only know the country where they first connected. We have to assume that that is their final destination
- if the person is not spanish, we assume their country of destination is their origin country. 
For trips that come from abroad a similar situation holds

This might create inaccuracies in the results


# 1. Format and eliminate unnecessary trips

In [2]:
# Trips during the week 22/09/2022 28/09/2022 (thursday to thursday)
# the day of study selected was Friday to put the air layer under pressure
all_trips = pd.read_csv(
    r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP4 Performance Assessment Solution\Demand data\Matrices MITMA\with_archetypes\20220922_28_trip_matrix_arc_pt_processed.csv.gz",
    compression="gzip",
    sep="|"
)

In [3]:
#Here there is no trip id but later trip id appears all the time
trips = all_trips[all_trips["date"] == 20220923].reset_index(drop=True).rename(columns={"origin_nut": "origin", "destination_nut": "destination"})
trips.head()

,date,trip_period,origin_zone,origin,origin_name,destination_zone,destination,destination_name,entry_point,exit_point,origin_purpose,destination_purpose,distance,route_distance,duration,mode,service,legs,trip_vehicle_type,nationality,home_census,home_zone,overnight_census,income,age,sex,vehicle_type,short_professional_driver,trips,trips_km,sample_trips,archetype_0,archetype_1,archetype_2,archetype_3,archetype_4,archetype_5,n_legs,mode_sequence,node_sequence,start_node,end_node,type,road_legs,train_legs,plane_legs,node_sequence_reduced,start_node_reduced,end_node_reduced
0,20220923,P00,01002,ES211,Álava,4802006,ES213,Vizcaya,NaN,NaN,NF,O,D04_[10000-50000),D04_[10000-50000),01-02,train,conv_unknown,P00*01002*01002*None*train_13121*00-01*road*No...,other,ES,2_48,4801303,2_48,I01_[10000-15000),A01_[25-45),male,passenger,False,4.135,139.516,1.0,0.000000,0.000000,2.067500,2.067500,0.000000,0.000000,3,road-train-road,train_13121-train_13200,train_13121,train_13200,national,2,1,0,train_13121-train_13200,train_13106,train_13200
1,20220923,P00,01009_AM,ES211,Álava,31010_AM,ES220,Navarra,NaN,NaN,O,H,D04_[10000-50000),D04_[10000-50000),00-01,train,conv_unknown,P00*01009_AM*01009_AM*None*train_11213*00-01*r...,other,ES,2_31,31010_AM,2_31,I01_[10000-15000),A01_[25-45),male,passenger,False,5.191,69.312,1.0,3.707857,0.000000,0.000000,0.494381,0.000000,0.988762,3,road-train-road,train_11213-train_11300,train_11213,train_11300,national,2,1,0,train_11213-train_11300,train_13106,train_11300
2,20220923,P00,01009_AM,ES211,Álava,abroad_208,abroad,abroad,NaN,ground_Fra_08,NF,NF,abroad,D05_[50000-inf),01-02,train,conv_unknown,P00*01009_AM*01009_AM*None*train_11213*00-01*r...,other,FR,NaN,NaN,NaN,NaN,NaN,NaN,passenger,False,1.599,309.588,1.0,0.888333,0.399750,0.142133,0.071067,0.062183,0.035533,3,road-train-road,train_11213-train_11600,train_11213,train_11600,international_D,2,1,0,train_11213-train_11600,train_11208,train_11511
3,20220923,P00,01036,ES211,Álava,4802006,ES213,Vizcaya,NaN,NaN,H,O,D04_[10000-50000),D04_[10000-50000),00-01,train,conv_unknown,P00*01036*01036*None*train_13106*00-01*road*No...,other,ES,2_01,01036,2_48,I02_[15000-inf),A02_[45-65),female,passenger,False,6.236,139.477,1.0,3.118000,0.000000,0.779500,1.559000,0.779500,0.000000,3,road-train-road,train_13106-train_13200,train_13106,train_13200,national,2,1,0,train_13106-train_13200,train_13106,train_13200
4,20220923,P00,0105902,ES211,Álava,09219,ES412,Burgos,NaN,NaN,O,NF,D04_[10000-50000),D04_[10000-50000),01-02,train,conv_unknown,P00*0105902*0105901*None*train_11208*00-01*roa...,other,ES,2_01,0105904,2_01,I02_[15000-inf),A02_[45-65),male,passenger,False,4.215,155.813,1.0,3.417568,0.113919,0.227838,0.227838,0.227838,0.000000,3,road-train-road,train_11208-train_11200,train_11208,train_11200,national,2,1,0,train_11208-train_11200,train_11208,train_11200


In [4]:
%load_ext autoreload

In [5]:
%autoreload
from script.trips_format import *

In [6]:
#associates each airport to the corresponding new NUTS
airports_to_NUTS={"airport_LPA":("ES705","Gran Canaria"),
                 "airport_FUE":("ES704","Fuerteventura"),
                 "airport_ACE":("ES708","Lanzarote"),
                 "airport_TFS":("ES709","Tenerife"),
                 "airport_TFN":("ES709","Tenerife"),
                 "airport_GMZ":("ES709","Tenerife"),
                 "airport_SPC":("ES707","La Palma"),
                 "airport_VDE":("ES703","El Hierro"),
                 "airport_PMI":("ES532","Mallorca"),
                 "airport_IBZ":("ES531","Eivissa i Formentera"),
                 "airport_MAH":("ES533","Menorca")}

In [7]:
trips=format_trips(trips,airports_to_NUTS)

17 columns were removed


First we remove the cercanías trips and the trips that go through weird stations

In [8]:
# remove cercanías
trips=trips[~(((trips["origin"]=="ES424")&(trips["destination"]=="ES300"))|((trips["origin"]=="ES300")&(trips["destination"]=="ES424")))]

In [9]:
# location of "ALL" train stops given by UiC
# However this list is still incomplete
stops_loc=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain+abroad\v=0.1\infrastructure\rail_info\stops.txt").astype(str) # everything is a string here to match other formatting
stops_loc["stop_id"] = stops_loc["stop_id"].apply(lambda x: "00" + x) #to make they start with 00

In [10]:
trips.loc[:,"weird_stations"] = trips["node_sequence_reduced"].apply(
    lambda x: find_weird_stations(x, stops_loc))

In [11]:
unique_weird_stations=get_weird_stations(trips["weird_stations"])

In [12]:
len(unique_weird_stations)

414

In [13]:
MobA_stations_coord=gpd.read_file(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain\v=0.7\oferta_transporte\train_stations\train_stations.shp")

In [14]:
# identifies all the un-localisable stations
nowhere_stations=set(unique_weird_stations)-set(MobA_stations_coord["ID"])
print(f"there are {len(nowhere_stations)} stations that are not in the data provided by MobA but appear in the trips dataframe")

there are 205 stations that are not in the data provided by MobA but appear in the trips dataframe


In [15]:
trips = trips[~trips["node_sequence_reduced"].apply(lambda x: any(station in x for station in nowhere_stations))]

Then we add the relevant information for international trips

In [16]:
international_codes=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain+abroad\v=0.1\infrastructure\countries mcc\mcc_to_nationality.txt", sep="|")
international_codes

,mcc,country_name,country
0,202,Greece,GR
1,204,Netherlands,NL
2,206,Belgium,BE
3,208,France,FR
4,212,Monaco,MC
...,...,...,...
227,746,Suriname,SR
228,748,Uruguay,UY
229,750,Falkland Islands (Malvinas),FK
230,901,International & Satellite Networks,-


In [17]:
mcc_to_country=international_codes.set_index("mcc")["country"].to_dict()

In [18]:
#it is better to divide trips into trips that go to or come from abroad and trips that do not come from
#or go to abroad to save time in computations

trips_abroad=trips[(trips["origin"]=="abroad")|(trips["destination"]=="abroad")]
trips_national=trips[~((trips["origin"]=="abroad")|(trips["destination"]=="abroad"))]

In [19]:
trips_abroad=format_trips_abroad(trips_abroad,mcc_to_country)

In [20]:
trips_national=format_trips_national(trips_national)

In [21]:
trips=pd.concat([trips_national,trips_abroad])

In [22]:
trips

,date,origin,origin_name,destination,destination_name,entry_point,exit_point,origin_purpose,destination_purpose,legs,nationality,archetype_0,archetype_1,archetype_2,archetype_3,archetype_4,archetype_5,n_legs,mode_sequence,node_sequence,start_node,end_node,type,road_legs,train_legs,plane_legs,node_sequence_reduced,start_node_reduced,end_node_reduced,mode_tp,trips
0,20220923,ES111,A Coruña,ES112,Lugo,NaN,NaN,H,NF,P06*15009*1501702*None*train_20402*00-01*road*...,ES,3.937095,0.051465,0.141530,0.424589,0.257326,0.192995,3,road-train-road,train_20402-train_20309,train_20402,train_20309,national,2,1,0,train_20402-train_20309,train_31400,train_20300,['rail'],5.005
1,20220923,ES111,A Coruña,ES112,Lugo,NaN,NaN,H,NF,P07*15005*1503009*None*train_20410*00-01*road*...,ES,3.351054,0.043805,0.120463,0.361388,0.219023,0.164267,3,road-train-road,train_20410-train_20309,train_20410,train_20309,national,2,1,0,train_20410-train_20309,train_31400,train_20300,['rail'],4.260
2,20220923,ES111,A Coruña,ES112,Lugo,NaN,NaN,H,NF,P07*1503007*1503004*None*train_31412*00-01*roa...,ES,4.821164,0.129254,0.064627,0.348985,0.413612,0.284358,3,road-train-road,train_31412-train_31400,train_31412,train_31400,national,2,1,0,train_31400,train_31400,train_31400,['rail'],6.062
3,20220923,ES111,A Coruña,ES112,Lugo,NaN,NaN,H,NF,P08*15008*15009*None*train_20400*00-01*road*No...,ES,2.562062,0.033491,0.092100,0.276301,0.167455,0.125591,3,road-train-road,train_20400-train_20305,train_20400,train_20305,national,2,1,0,train_20400-train_20305,train_31400,train_20300,['rail'],3.257
4,20220923,ES111,A Coruña,ES112,Lugo,NaN,NaN,H,NF,P08*1503009*1503004*None*train_31412*00-01*roa...,ES,7.604907,0.241774,0.109897,0.527508,0.615426,0.549487,3,road-train-road,train_31412-train_31400,train_31412,train_31400,national,2,1,0,train_31400,train_31400,train_31400,['rail'],9.649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77395,20220923,ZA,ZA,ES300,Madrid,airport_MAD,NaN,NF,NF,P19*abroad_655*2807921*None*airport_MAD*00-01*...,AR,5.411086,1.365160,1.625308,0.066895,0.304745,0.076806,2,plane-road,airport_MAD,airport_MAD,airport_MAD,international_O,1,0,1,airport_MAD,airport_MAD,airport_MAD,['air'],8.850
77396,20220923,ZA,ZA,ES511,Barcelona,airport_BCN,NaN,NF,NF,P07*abroad_655*0816904*None*airport_BCN*00-01*...,AR,1.372793,1.028286,0.975929,0.048168,0.104713,0.069111,4,plane-road-train-road,airport_BCN-train_72400-train_71801,airport_BCN,train_71801,international_O,2,1,1,airport_BCN,airport_BCN,airport_BCN,"['air', 'rail']",3.599
77397,20220923,ZA,ZA,ES511,Barcelona,airport_BCN,NaN,NF,NF,P13*abroad_655*0816904*None*airport_BCN*00-01*...,ZA,1.372793,1.028286,0.975929,0.048168,0.104713,0.069111,2,plane-road,airport_BCN,airport_BCN,airport_BCN,international_O,1,0,1,airport_BCN,airport_BCN,airport_BCN,['air'],3.599
77398,20220923,ZA,ZA,ES521,Alicante / Alacant,ground_Por_00,NaN,NF,NF,P15*abroad_655*3205405*None*train_22100*00-01*...,AR,0.992828,2.109759,0.496414,0.000000,0.000000,0.000000,3,road-train-road,train_22100-train_60600,train_22100,train_60600,international_O,2,1,0,train_22100-train_60600,train_22100,train_60600,['rail'],3.599


# 2. Analysis of trips going abroad

We can distinguish the region in France and Portugal where the trip finishes, however we do not have the same level of detail with the rest of the countries, so we want to group those trips. 

In [25]:
trips_from_abroad=trips[~(trips["origin"].str.startswith("ES"))]

In [27]:
trips_to_abroad=trips[~(trips["destination"].str.startswith("ES"))]

In [28]:
(trips_abroad["trips"].sum()/trips["trips"].sum())*100

np.float64(33.04858065861245)

33.05% of trips from my matrix either come from abroad or go abroad

In [31]:
trips_to_abroad[trips_to_abroad["destination"]=="DE"]["trips"].sum()

np.float64(4445.545)

In [32]:
outgoing_grouped=trips_to_abroad.groupby("destination")["trips"].sum().reset_index()

In [33]:
incoming_grouped=trips_from_abroad.groupby("origin")["trips"].sum().reset_index()

In [38]:
outgoing_grouped=outgoing_grouped.sort_values(by="trips",ascending=False)

In [36]:
incoming_grouped=incoming_grouped.sort_values(by="trips",ascending=False)

In [39]:
outgoing_grouped

,destination,trips
36,FR,58908.511
85,PT,28366.809
37,GB,13100.902
105,US,4482.920
29,DE,4445.545
...,...,...
99,TD,3.599
98,SY,3.599
111,ZW,3.599
67,MK,1.587


In [40]:
incoming_grouped

,origin,trips
37,FR,48896.216
84,PT,22186.364
38,GB,14478.282
50,IT,7282.493
29,DE,5934.856
...,...,...
53,KH,3.599
101,TT,3.599
97,SY,3.599
105,UZ,3.599
